In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import matplotlib.pyplot as plt
import os 
os.chdir(r"C:\Training\Academy\Statistics (Python)\Cases\Glass Identification")
from sklearn.preprocessing import LabelEncoder 
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [2]:
glass = pd.read_csv("Glass.csv")
glass['Type'].unique()

array(['building_windows_float_processed',
       'building_windows_non_float_processed',
       'vehicle_windows_float_processed', 'containers', 'tableware',
       'headlamps'], dtype=object)

In [3]:
glass.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,building_windows_float_processed


In [4]:
dum_gls = pd.get_dummies(glass)
dum_gls.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type_building_windows_float_processed,Type_building_windows_non_float_processed,Type_containers,Type_headlamps,Type_tableware,Type_vehicle_windows_float_processed
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1,0,0,0,0,0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1,0,0,0,0,0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1,0,0,0,0,0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1,0,0,0,0,0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1,0,0,0,0,0


In [5]:
X = dum_gls.iloc[:,:-6]
y = dum_gls.iloc[:,-6:]
y.head()

,Type_building_windows_float_processed,Type_building_windows_non_float_processed,Type_containers,Type_headlamps,Type_tableware,Type_vehicle_windows_float_processed
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [6]:
y = y.values
y.shape

(214, 6)

In [7]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                   random_state=24,stratify=glass['Type'])
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [8]:
X_test.shape, y_test.shape

((65, 9), (65, 6))

In [9]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([149, 9])
torch.Size([149, 6])


In [10]:
type(X_torch)

torch.Tensor

In [11]:
X_scl_trn.shape[1]

9

#### Instantiate the `TensorDataset`

In [12]:
joint_dataset = TensorDataset(X_torch.float(), y_torch.float())

#### Data Loader

In [13]:
torch.manual_seed(24)
data_loader = DataLoader(joint_dataset, batch_size=16,shuffle=True,
                         drop_last=False)

### Model Definition

In [14]:
torch.manual_seed(24)
class MLPClassifier(torch.nn.Module):    
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(in_features=num_features, out_features=15)
        self.linear2 = nn.Linear(in_features=15, out_features=8)
        self.linear3 = nn.Linear(in_features=8, out_features=6)
        self.act1 = nn.SELU()
        self.dropout = nn.Dropout(0.3)
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x) 
        output = self.linear3(x)
        return output

In [15]:
model = MLPClassifier(num_features = X_scl_trn.shape[1])

In [16]:
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 15]             150
           Dropout-2                [-1, 1, 15]               0
              SELU-3                [-1, 1, 15]               0
            Linear-4                 [-1, 1, 8]             128
              ReLU-5                 [-1, 1, 8]               0
            Linear-6                 [-1, 1, 6]              54
Total params: 332
Trainable params: 332
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.1)

Generating predictions with default weights

In [18]:
y_pred = model(X_torch.float())
y_pred[:3]

tensor([[-0.0372, -0.1083,  0.2224, -0.2943,  0.3321,  0.1703],
        [-0.1372, -0.0409,  0.1656, -0.3878,  0.2538,  0.2478],
        [-0.1156, -0.1052,  0.1654, -0.3457,  0.3441,  0.2353]],
       grad_fn=<SliceBackward0>)

In [19]:
print(y_torch.shape)

torch.Size([149, 6])


In [20]:
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([149, 6])
torch.Size([149, 6])


### Initial Log Loss

In [21]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(1.9168, grad_fn=<DivBackward1>)

In [22]:
for epoch in np.arange(0,500):
    for i, batch in enumerate(data_loader, 1):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred_prob = model(batch[0].float())

      # Compute and print loss
      loss = criterion(y_pred_prob, batch[1].float())

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()

      # perform a backward pass (backpropagation)
      loss.backward()

      # Update the parameters
      optimizer.step()
    
    if epoch%100 == 0:
          print('epoch: ', epoch+1,' train loss: ', loss.item())

epoch:  1  train loss:  1.2292498350143433
epoch:  101  train loss:  0.7457834482192993
epoch:  201  train loss:  0.6400514841079712
epoch:  301  train loss:  0.4183969497680664
epoch:  401  train loss:  0.5487287640571594


### Training Set Log Loss

In [23]:
loss

tensor(0.6801, grad_fn=<DivBackward1>)

In [24]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [25]:
y_wt_sum = model(X_torch_test.float()) 
softmax = nn.Softmax(dim=1)
pred_proba = softmax(y_wt_sum)
pred_proba

tensor([[7.9264e-01, 1.0527e-01, 2.3254e-04, 8.9085e-08, 4.1496e-05, 1.0182e-01],
        [1.4050e-27, 5.4203e-07, 3.1321e-07, 1.0000e+00, 9.8856e-11, 8.4111e-30],
        [2.5006e-01, 5.9942e-01, 2.6810e-04, 8.7735e-04, 1.2324e-03, 1.4814e-01],
        [7.1076e-01, 7.0876e-02, 4.8321e-05, 3.2650e-07, 4.1337e-04, 2.1790e-01],
        [7.1736e-01, 4.9334e-02, 2.4538e-03, 3.6557e-06, 3.6998e-02, 1.9385e-01],
        [4.7164e-01, 3.9049e-01, 1.5090e-07, 6.9381e-08, 2.1068e-08, 1.3787e-01],
        [6.2587e-09, 6.2392e-02, 6.0333e-02, 9.2028e-03, 8.6807e-01, 3.2457e-10],
        [1.6377e-01, 8.1824e-01, 1.2927e-06, 7.1792e-07, 1.1386e-07, 1.7989e-02],
        [6.1104e-01, 1.3830e-01, 8.1996e-05, 2.7242e-04, 1.5564e-03, 2.4875e-01],
        [4.5682e-01, 3.7607e-01, 3.6318e-06, 1.2234e-07, 1.6893e-07, 1.6711e-01],
        [7.2670e-16, 3.1144e-02, 9.7804e-04, 9.6785e-01, 2.4229e-05, 1.5338e-17],
        [7.5646e-04, 1.0398e-01, 8.9098e-04, 6.3064e-01, 2.6358e-01, 1.4420e-04],
        [4.2551e

In [26]:
pred_proba.shape

torch.Size([65, 6])

`detach().numpy()` converts `torch.Tensor` into `numpy` object

In [27]:
y_pred = np.argmax(pred_proba.detach().numpy(), axis=1 )
y_pred

array([0, 3, 1, 0, 0, 0, 4, 1, 0, 0, 3, 3, 1, 0, 1, 3, 3, 0, 1, 1, 0, 3,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 3, 0, 4, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       3, 3, 0, 1, 0, 2, 0, 0, 2, 0, 1, 0, 3, 0, 0, 1, 0, 0, 2, 0, 1],
      dtype=int64)

In [28]:
y_test_lbl = np.argmax(y_test, axis=1)
y_test_lbl[:5]

array([1, 3, 0, 0, 0], dtype=int64)

### Test Set Accuracy Score

In [29]:
print(accuracy_score(y_test_lbl,y_pred))

0.6461538461538462


In [30]:
pred_proba.shape

torch.Size([65, 6])

### Test Set Log Loss

In [31]:
log_loss(y_test_lbl, pred_proba.detach().numpy())

1.258572856843681